# Classes Desbalanceadas

Classes desbalanceadas referem-se a uma situação em problemas de classificação onde o número de observações não é igual (ou quase igual) entre as classes. Em outras palavras, uma das classes tem muito mais exemplos que a outra. Isso é comum em muitos cenários do mundo real.

Por exemplo, considere um dataset sobre detecção de fraudes em transações bancárias. Em milhares de transações, apenas uma pequena fração pode ser fraudulenta, enquanto a grande maioria das transações é legítima. Este é um exemplo clássico de classes desbalanceadas.

## Criando um dataset de exemplo

In [9]:
import pandas as pd
import numpy as np

# Tamanho do dataset
num_samples = 100000

# Semente aleatória para reprodutibilidade
np.random.seed(42)

# Gerar dados fictícios
valor_transacao = np.random.randint(1, 10000, num_samples)
idade_titular = np.random.randint(18, 80, num_samples)
num_transacoes_24h = np.random.poisson(lam=5, size=num_samples)  # Usando uma distribuição de Poisson para modelar a contagem
eh_internacional = np.random.choice([0, 1], size=num_samples, p=[0.9, 0.1])  # 10% das transações são internacionais

# Supor que transações com valores muito altos, feitas internacionalmente e por titulares muito jovens têm maior probabilidade de serem fraudes
indices_fraudes = np.where((valor_transacao > 8000) & (eh_internacional == 1) & (idade_titular < 25))[0]
fraude = np.zeros(num_samples)
fraude[indices_fraudes] = 1

# Se quisermos um desbalanceamento ainda maior, podemos amostrar aleatoriamente 1% dos índices de fraude
#fraude_indices = np.random.choice(indices_fraudes, size=int(0.01 * len(indices_fraudes)), replace=False)
#fraude = np.zeros(num_samples)
#fraude[fraude_indices] = 1

# Criando o DataFrame
df = pd.DataFrame({
    'Valor da Transação': valor_transacao,
    'Idade do Titular': idade_titular,
    'Num Transações 24h': num_transacoes_24h,
    'É Internacional': eh_internacional,
    'Fraude': fraude
})

print(df['Fraude'].value_counts())
print(df.head())

Fraude
0.0    99757
1.0      243
Name: count, dtype: int64
   Valor da Transação  Idade do Titular  Num Transações 24h  É Internacional  \
0                7271                73                   7                0   
1                 861                18                   7                0   
2                5391                54                   4                0   
3                5192                43                   3                0   
4                5735                67                   6                0   

   Fraude  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  


## As métricas de desempenho

**Bibliotecas**

In [4]:
# importa as bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_score, recall_score

**Divisão do dataset em treino e teste**

In [5]:
X = df.drop('Fraude', axis=1)
y = df['Fraude']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**Treina um modelo de regressão logística**

In [6]:
# Treinar um modelo de regressão logística
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

**Prevendo no conjunto de testes**

In [14]:
y_pred = model.predict(X_test)

**Calculando as métricas**

In [15]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Revocação (Recall):", recall)
print("F1-score:", f1)

Acurácia: 0.9992333333333333
Precisão: 0.8840579710144928
Revocação (Recall): 0.8026315789473685
F1-score: 0.8413793103448277
